In [12]:

# %pip install pandas numpy scikit-learn matplotlib seaborn tensorflow keras xgboost


First I tried a simple 2 layer CNN model, consisting of two convolutional blocks (3→32, 32→64) each with ReLU and max pooling, flattenning the resulting 64×8×8 feature map into a 4096 dimensional vector, and applies a single dropout layer in the fully connected head. Its design makes it fast and lightweight, but without batch normalization or extra layers it can’t learn as deepand well regularized features as a deeper network would.So I moved on to a 3layer CNN. This three layer CNN adds a third convolutional block (with 3→32→64→128 filters), each including batch normalization, activation, pooling, and dropout, to progressively learn richer image features down to a 128×4×4 map. It then flattens these 2048 features into a 128 unit dense layer (with dropout) before the final three way classification.

In [ ]:

import os
import pandas as pd
import numpy as np

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms

#Read training and test ssets
DATA_DIR = "data"

train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df  = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

print("Train:", train_df.shape, "Test:", test_df.shape)

# Check for missing values
assert train_df.isna().sum().sum() == 0, "Missing in train!"
assert test_df .isna().sum().sum() == 0, "Missing in test!"

# Separate IDs and labels
train_ids = train_df["id"].values
## zero-based for PyTorch
train_labels = train_df["y"].values.astype(np.int64) - 1  
train_df = train_df.drop(columns=["id", "y"])
test_ids = test_df["id"].values
test_df = test_df.drop(columns=["id"])

# Normalize pixel intensities(it is already in [0,1] but we cast to float32)
train_pixels = train_df.values.astype(np.float32)
test_pixels  = test_df.values .astype(np.float32)



FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data\\train.csv'

In [ ]:
#  Each row is [r3132, g0101, b0101], so we reshape to (N, 3, 32, 32)
def to_image_array(flat_array):
    # shape (N, 3072) → (N, 3, 32, 32)
    return flat_array.reshape(-1, 3, 32, 32)

X_train = to_image_array(train_pixels)
X_test  = to_image_array(test_pixels)


In [ ]:
class FarmImageDataset(Dataset):
    def __init__(self, images, labels=None):
        self.images = torch.from_numpy(images)     
        self.labels = None if labels is None else torch.from_numpy(labels)
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        x = self.images[idx]
        if self.labels is None:
            return x
        y = self.labels[idx]
        return x, y

# Split train into train/validation

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, train_labels, test_size=0.2, stratify=train_labels, random_state=42
)

train_ds = FarmImageDataset(X_tr, y_tr)
val_ds   = FarmImageDataset(X_val, y_val)
test_ds  = FarmImageDataset(X_test)

batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size)
test_loader  = DataLoader(test_ds,  batch_size=batch_size)


In [ ]:
##simple CNN model

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),    #   → (32,32,32)
            nn.ReLU(),
            nn.MaxPool2d(2),                               #  → (32,16,16)
            nn.Conv2d(32, 64, kernel_size=3, padding=1),    #  → (64,16,16)
            nn.ReLU(),
            nn.MaxPool2d(2),                               # →  (64,8,8)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),                                  #  →  (64*8*8 = 4096)
            nn.Linear(64*8*8, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_epoch(loader):
    model.train()
    total_loss, total_correct = 0, 0
    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        preds = model(Xb)
        loss = criterion(preds, yb)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        total_loss += loss.item() * Xb.size(0)
        total_correct += (preds.argmax(1) == yb).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

def eval_epoch(loader):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            preds = model(Xb)
            total_loss += criterion(preds, yb).item() * Xb.size(0)
            total_correct += (preds.argmax(1) == yb).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

best_val_acc = 0
for epoch in range(1, 26):
    train_loss, train_acc = train_epoch(train_loader)
    val_loss, val_acc     = eval_epoch(val_loader)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_cnn.pth")
    print(f"Epoch {epoch:02d} – train_acc: {train_acc:.3f}, val_acc: {val_acc:.3f}")


Epoch 01 – train_acc: 0.633, val_acc: 0.817
Epoch 02 – train_acc: 0.768, val_acc: 0.838
Epoch 03 – train_acc: 0.833, val_acc: 0.875
Epoch 04 – train_acc: 0.874, val_acc: 0.887
Epoch 05 – train_acc: 0.910, val_acc: 0.904
Epoch 06 – train_acc: 0.903, val_acc: 0.908
Epoch 07 – train_acc: 0.914, val_acc: 0.908
Epoch 08 – train_acc: 0.922, val_acc: 0.921
Epoch 09 – train_acc: 0.918, val_acc: 0.900
Epoch 10 – train_acc: 0.894, val_acc: 0.904
Epoch 11 – train_acc: 0.884, val_acc: 0.929
Epoch 12 – train_acc: 0.941, val_acc: 0.946
Epoch 13 – train_acc: 0.954, val_acc: 0.963
Epoch 14 – train_acc: 0.963, val_acc: 0.954
Epoch 15 – train_acc: 0.950, val_acc: 0.942
Epoch 16 – train_acc: 0.948, val_acc: 0.921
Epoch 17 – train_acc: 0.963, val_acc: 0.958
Epoch 18 – train_acc: 0.967, val_acc: 0.933
Epoch 19 – train_acc: 0.968, val_acc: 0.971
Epoch 20 – train_acc: 0.974, val_acc: 0.933
Epoch 21 – train_acc: 0.953, val_acc: 0.950
Epoch 22 – train_acc: 0.974, val_acc: 0.946
Epoch 23 – train_acc: 0.975, val

In [ ]:
# Load best model
model.load_state_dict(torch.load("best_cnn.pth"))
model.eval()


# Predict on test set
all_preds = []
with torch.no_grad():
    for Xb in test_loader:
        Xb = Xb.to(device)
        preds = model(Xb).argmax(1).cpu().numpy() + 1 
        all_preds.append(preds)
all_preds = np.concatenate(all_preds)

# generaate CSV
submission_file = pd.DataFrame({"id": test_ids, "y": all_preds})
submission_file.to_csv("cnn_2.csv", index=False)
print("saved cnn_2.csv with", len(submission_file), "rows.")


saved cnn_2.csv with 1200 rows.


In [ ]:
# training loop 
best_val_acc = 0
for epoch in range(1, 21):
    train_loss, train_acc = train_epoch(train_loader)
    val_loss, val_acc     = eval_epoch(val_loader)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_cnn.pth")
    print(f"Epoch {epoch:02d} – train_acc: {train_acc:.3f}, val_acc: {val_acc:.3f}")


print(f"\nBest validation accuracy achieved: {best_val_acc*100:.2f}%")
# Load best weights and reevaluate
model.load_state_dict(torch.load("best_cnn.pth"))
val_loss, val_acc = eval_epoch(val_loader)
print(f"Final validation accuracy (re-loaded best model): {val_acc*100:.2f}%")


Epoch 01 – train_acc: 0.969, val_acc: 0.946
Epoch 02 – train_acc: 0.969, val_acc: 0.942
Epoch 03 – train_acc: 0.963, val_acc: 0.954
Epoch 04 – train_acc: 0.957, val_acc: 0.954
Epoch 05 – train_acc: 0.976, val_acc: 0.942
Epoch 06 – train_acc: 0.975, val_acc: 0.967
Epoch 07 – train_acc: 0.975, val_acc: 0.963
Epoch 08 – train_acc: 0.974, val_acc: 0.967
Epoch 09 – train_acc: 0.978, val_acc: 0.963
Epoch 10 – train_acc: 0.981, val_acc: 0.942
Epoch 11 – train_acc: 0.974, val_acc: 0.963
Epoch 12 – train_acc: 0.983, val_acc: 0.938
Epoch 13 – train_acc: 0.984, val_acc: 0.963
Epoch 14 – train_acc: 0.985, val_acc: 0.929
Epoch 15 – train_acc: 0.980, val_acc: 0.963
Epoch 16 – train_acc: 0.983, val_acc: 0.946
Epoch 17 – train_acc: 0.994, val_acc: 0.942
Epoch 18 – train_acc: 0.989, val_acc: 0.967
Epoch 19 – train_acc: 0.990, val_acc: 0.963
Epoch 20 – train_acc: 0.994, val_acc: 0.942

Best validation accuracy achieved: 96.67%
Final validation accuracy (re-loaded best model): 96.67%


I then moved on to a 3layer CNN model; This three layer CNN adds a third convolutional block (with 3→32→64→128 filters), each including batch normalization, activation, pooling, and dropout, to progressively learn richer image features down to a 128×4×4 map. It then flattens these 2048 features into a 128 unit dense layer (with dropout) before the final three way classification.:

In [ ]:
#### 3 layer CNN:
import os
import pandas as pd
import numpy as np

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms


#Read training and test ssets
DATA_DIR = "data"
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df  = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
print("Train:", train_df.shape, "Test:", test_df.shape)

# Check for missing values
assert train_df.isna().sum().sum() == 0, "Missing in train!"
assert test_df .isna().sum().sum() == 0, "Missing in test!"


# Extract ids & labels
train_ids    = train_df.pop("id").values
## zero-based for PyTorch
train_labels = (train_df.pop("y").values - 1).astype(np.int64)  
test_ids     = test_df.pop("id").values

# Convert to float32 and reshape to (N,3,32,32)

X_train = train_df.values.astype(np.float32).reshape(-1, 3, 32, 32)
X_test  = test_df.values.astype(np.float32).reshape(-1, 3, 32, 32)

Train: (1200, 3074) Test: (1200, 3073)


In [ ]:

# DataLoader
class FarmDataset(Dataset):
    def __init__(self, images, labels=None):
        self.images = torch.from_numpy(images)
        self.labels = None if labels is None else torch.from_numpy(labels)
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        x = self.images[idx]
        if self.labels is None:
            return x
        y = self.labels[idx]
        return x, y

# Train/validation split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, train_labels,
    test_size=0.2,
    stratify=train_labels,
    random_state=42
)

batch_size   = 64
train_loader = DataLoader(FarmDataset(X_tr,  y_tr), batch_size, shuffle=True)
val_loader   = DataLoader(FarmDataset(X_val, y_val), batch_size)
test_loader  = DataLoader(FarmDataset(X_test),    batch_size)

# Define the 3-layer CNN
class ThreeLayerCNN(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1: 3→32
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),            ### 32×32×32 → 32×16×16
            nn.Dropout(0.25),

            # Block 2: 32→64
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),              ##### 64×16×16 → 64×8×8
            nn.Dropout(0.25),

            # Block 3: 64→128
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),            #### 128×8×8 → 128×4×4
            nn.Dropout(0.25),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),                  ####→ 128*4*4 = 2048
            nn.Linear(2048, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# training setup
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = ThreeLayerCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training and evaluation functions
def train_epoch(loader):
    model.train()
    total_loss = total_correct = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        preds  = model(xb)
        loss   = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss   += loss.item() * xb.size(0)
        total_correct+= (preds.argmax(1) == yb).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

def eval_epoch(loader):
    model.eval()
    total_loss = total_correct = 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            preds  = model(xb)
            total_loss   += criterion(preds, yb).item() * xb.size(0)
            total_correct+= (preds.argmax(1) == yb).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

#  Train for 30 epochs, save  the best model
best_val_acc = 0.0
for epoch in range(1, 31):
    tr_loss, tr_acc = train_epoch(train_loader)
    vl_loss, vl_acc = eval_epoch(val_loader)
    if vl_acc > best_val_acc:
        best_val_acc = vl_acc
        torch.save(model.state_dict(), "best_three_layer_cnn.pth")
    print(f"Epoch {epoch:02d} | train_acc: {tr_acc:.3f} | val_acc: {vl_acc:.3f}")

print(f"\nBest validation accuracy: {best_val_acc*100:.2f}%")

# Reload best modeln and final validation
model.load_state_dict(torch.load("best_three_layer_cnn.pth"))
_, final_acc = eval_epoch(val_loader)
print(f"Reloaded best model val_acc: {final_acc*100:.2f}%")

# generate CSV
model.eval()
all_preds = []
with torch.no_grad():
    for xb in test_loader:
        xb    = xb.to(device)
        preds = model(xb).argmax(1).cpu().numpy() + 1  
        all_preds.append(preds)
all_preds = np.concatenate(all_preds)

submission = pd.DataFrame({
    "id": test_ids,
    "y":  all_preds
})
submission.to_csv("3L_cnn.csv", index=False)
print("Saved 3L_cnn.csv with", len(submission), "rows.")


Epoch 01 | train_acc: 0.647 | val_acc: 0.338
Epoch 02 | train_acc: 0.883 | val_acc: 0.688
Epoch 03 | train_acc: 0.920 | val_acc: 0.817
Epoch 04 | train_acc: 0.934 | val_acc: 0.971
Epoch 05 | train_acc: 0.954 | val_acc: 0.921
Epoch 06 | train_acc: 0.941 | val_acc: 0.938
Epoch 07 | train_acc: 0.948 | val_acc: 0.879
Epoch 08 | train_acc: 0.948 | val_acc: 0.921
Epoch 09 | train_acc: 0.946 | val_acc: 0.963
Epoch 10 | train_acc: 0.954 | val_acc: 0.975
Epoch 11 | train_acc: 0.970 | val_acc: 0.958
Epoch 12 | train_acc: 0.967 | val_acc: 0.954
Epoch 13 | train_acc: 0.959 | val_acc: 0.933
Epoch 14 | train_acc: 0.977 | val_acc: 0.958
Epoch 15 | train_acc: 0.974 | val_acc: 0.904
Epoch 16 | train_acc: 0.983 | val_acc: 0.954
Epoch 17 | train_acc: 0.978 | val_acc: 0.863
Epoch 18 | train_acc: 0.942 | val_acc: 0.879
Epoch 19 | train_acc: 0.971 | val_acc: 0.929
Epoch 20 | train_acc: 0.984 | val_acc: 0.979
Epoch 21 | train_acc: 0.984 | val_acc: 0.929
Epoch 22 | train_acc: 0.990 | val_acc: 0.971
Epoch 23 |